In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

from JacobianODE.jacobians.jacobian_utils import load_config
from JacobianODE.jacobians.run_jacobians import train_jacobians

In [3]:
# FILL THIS IN FOR YOUR OWN USE
save_dir = "/orcd/data/ekmiller/001/eisenaj/JacobianODE/lightning"
wandb_entity = "chaotic-consciousness"

## Setup

In [4]:
# load the config
overrides = [
    "data=dysts",
    "data.flow._target_=JacobianODE.dysts_sim.flows.Lorenz",
    "data.postprocessing.obs_noise=0.01",
    "training.lightning.loop_closure_weight=0.001",
    f"training.logger.save_dir=./output",
    # f"wandb_entity={wandb_entity}",
]
cfg = load_config(overrides=overrides)

In [6]:
train_jacobians(cfg)

Run MLP__hidden_dim_[256, 1024, 2048, 2048]__num_layers_4__residuals_False__dropout_0.0__activation_silu__batch_size_16__direct_True__loss_func_mse__alpha_hal_0.1__l2_penalty_0__l1_penalty_0.0__obs_noise_scale_0.0000__final_obs_noise_scale_0__y0_noise_scale_0__noise_annealing_False__log_interval_100__alpha_teacher_forcing_1__teacher_forcing_annealing_True__gamma_teacher_forcing_0.999__teacher_forcing_update_interval_5__teacher_forcing_steps_1__min_alpha_teacher_forcing_0__alpha_validation_0__obs_noise_scale_validation_0.0000__loss_func_validation_mse__traj_init_steps_validation_15__inner_N_validation_20__data_type_dysts__jacobianODEint_kwargs_{'traj_init_steps': 15, 'inner_path': 'line', 'inner_N': 20, 'interp_pts': 4}__gradient_clip_val_1.0__gradient_clip_algorithm_norm__optimizer_AdamW__optimizer_kwargs_{'lr': 0.0001, 'weight_decay': 0.0001}__use_scheduler_True__min_lr_1e-06__k_scale_1__jac_penalty_0.0__jac_norm_ord_fro__loop_closure_training_True__mix_trajectories_True__loop_closure

/Users/mahdiyarshahbazi/miniconda3/envs/jacobian/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


ValueError: You set `strategy=ddp_notebook` but strategies from the DDP family are not supported on the MPS accelerator. Either explicitly set `accelerator='cpu'` or change the strategy.